## Setup 🪓

In [1]:
# CELL 1: Install + GPU
import torch
print("GPU:", torch.cuda.is_available())

!pip install diffusers transformers accelerate peft -q

GPU: True


In [2]:
# CELL 2: Upload data folder
from google.colab import files
import os

uploaded = files.upload()  # ← Drag & drop your `data.zip` here

# Unzip
!unzip -q data.zip
print("Data uploaded!")

Saving data.zip to data.zip
Data uploaded!


In [7]:
# FINAL FIX: Install diffusers from SOURCE (0.36.0+ dev version)
!pip uninstall -y diffusers
!pip install -q git+https://github.com/huggingface/diffusers

# Reinstall deps
!pip install -q transformers accelerate peft datasets

# Download LATEST LoRA script (compatible)
!rm -f train_text_to_image_lora.py
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora.py

print("INSTALLED DEV VERSION — READY FOR TRAINING!")

Found existing installation: diffusers 0.35.2
Uninstalling diffusers-0.35.2:
  Successfully uninstalled diffusers-0.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INSTALLED DEV VERSION — READY FOR TRAINING!


In [9]:
# CELL: Convert captions.json → metadata.csv (required by LoRA script)
import json
import pandas as pd
from pathlib import Path

# Load your captions
with open("data/captions.json") as f:
    data = json.load(f)

# Create DataFrame: file_name, caption
rows = []
for item in data:
    # Extract filename from image_path
    img_path = item["image_path"]
    filename = Path(img_path).name  # e.g., "meme_0.png"
    caption = item["caption"]
    rows.append({"file_name": filename, "text": caption})

df = pd.DataFrame(rows)
df.to_csv("data/metadata.csv", index=False)

print("metadata.csv created! Ready for training.")

metadata.csv created! Ready for training.


In [11]:
# FINAL FIX: Create metadata.csv INSIDE images/ folder
import json
import pandas as pd
import os

# Load your captions
with open("data/captions.json") as f:
    data = json.load(f)

# Build rows
rows = []
for item in data:
    filename = item["image_path"].split("/")[-1]  # e.g., "meme_0.png"
    caption = item["caption"]
    rows.append({"file_name": filename, "text": caption})

# Save CSV INSIDE images/ folder
df = pd.DataFrame(rows)
csv_path = "data/images/metadata.csv"
df.to_csv(csv_path, index=False)

print(f"metadata.csv created at: {csv_path}")
print("READY FOR TRAINING!")

metadata.csv created at: data/images/metadata.csv
READY FOR TRAINING!


In [13]:
# FINAL TRAINING CELL — 100% WORKING
!accelerate launch train_text_to_image_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir="data/images" \
  --caption_column="text" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=200 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --checkpointing_steps=100 \
  --output_dir="lora_meme_adapter" \
  --validation_prompt="Drake meme, 'ME' top, 'DEBUGGING' bottom, bold text" \
  --report_to="tensorboard" \
  --mixed_precision="fp16" \
  --seed=42

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-08 08:23:17.761599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762590197.927345   11073 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762590197.979519   11073 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762590198.336891   11073 computation_plac

In [14]:
# DOWNLOAD YOUR CUSTOM MEME LoRA
from google.colab import files
!zip -r lora_meme_adapter.zip lora_meme_adapter
files.download("lora_meme_adapter.zip")

  adding: lora_meme_adapter/ (stored 0%)
  adding: lora_meme_adapter/checkpoint-100/ (stored 0%)
  adding: lora_meme_adapter/checkpoint-100/scaler.pt (deflated 64%)
  adding: lora_meme_adapter/checkpoint-100/optimizer.bin (deflated 8%)
  adding: lora_meme_adapter/checkpoint-100/model.safetensors (deflated 8%)
  adding: lora_meme_adapter/checkpoint-100/random_states_0.pkl (deflated 27%)
  adding: lora_meme_adapter/checkpoint-100/scheduler.bin (deflated 61%)
  adding: lora_meme_adapter/checkpoint-100/pytorch_lora_weights.safetensors (deflated 6%)
  adding: lora_meme_adapter/checkpoint-200/ (stored 0%)
  adding: lora_meme_adapter/checkpoint-200/scaler.pt (deflated 64%)
  adding: lora_meme_adapter/checkpoint-200/optimizer.bin (deflated 8%)
  adding: lora_meme_adapter/checkpoint-200/model.safetensors (deflated 8%)
  adding: lora_meme_adapter/checkpoint-200/random_states_0.pkl (deflated 27%)
  adding: lora_meme_adapter/checkpoint-200/scheduler.bin (deflated 61%)
  adding: lora_meme_adapter/c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>